<a href="https://colab.research.google.com/github/marekk1717/letsautomate/blob/master/Commvault_Python_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## **Installation of cvpysdk and required dependencies**

In [0]:
!pip install xmltodict future requests beautifultable
!pip install git+https://github.com/CommvaultEngg/cvpysdk.git@dev

In [0]:
from cvpysdk.commcell import Commcell
import getpass
import json
from beautifultable import BeautifulTable
import pandas as pd
import random
import requests



---
<br><br>
## **Connect to WebConsole**

In [0]:
WEBCONSOLE_URL = input('WebConsole URL: ')
COMMCELL_USERNAME = input('Username: ')
COMMCELL_PASSWORD = getpass.getpass(prompt='Password: ')

commcell = Commcell(webconsole_hostname=WEBCONSOLE_URL, commcell_username=COMMCELL_USERNAME, commcell_password=COMMCELL_PASSWORD)
print('Connected to ' + commcell.commserv_hostname + ', Version: v11SP' + str(commcell.commserv_version))



---
<br><br>

## **List of active backup jobs**

In [0]:
active_backup_jobs = commcell.job_controller.active_jobs(limit=1000, job_filter='Backup', job_summary='Full')
table = BeautifulTable(max_width=300)
table.column_headers = ["JOB ID", "Client", "AppType", "% Complete", 'AppSize (GB)']
for job in active_backup_jobs:
  table.append_row([job, active_backup_jobs[job]['destinationClient']['clientName'], active_backup_jobs[job]['appTypeName'], 
                    active_backup_jobs[job]['percentComplete'], active_backup_jobs[job]['sizeOfApplication']/1024/1024/1024])
print(table)

---
<br><br>
## **Completed backup jobs (1h)**

In [0]:
completed_backup_jobs = commcell.job_controller.finished_jobs(limit=1000, job_filter='Backup', job_summary='Full', lookup_time=1)
table = BeautifulTable(max_width=300)
table.column_headers = ["JOB ID", "Client", "AppType", 'AppSize (GB)', 'Media Size (GB)', 'Status']
for job in completed_backup_jobs:
  table.append_row([job, completed_backup_jobs[job]['destinationClient']['clientName'], completed_backup_jobs[job]['appTypeName'], 
                    completed_backup_jobs[job]['sizeOfApplication']/1024/1024/1024, completed_backup_jobs[job]['sizeOfMediaOnDisk']/1024/1024/1024, 
                    completed_backup_jobs[job]['status']])
print(table)



---
<br><br>

## **Jobs stats (last 24h)**

In [0]:
completed_backup_jobs = commcell.job_controller.finished_jobs(limit=1000, job_filter='Backup', job_summary='Full', lookup_time=24)
df = pd.DataFrame(columns=['jobid', 'client', 'app_type', 'app_size', 'media_size', 'status'])
a = 0
for job in completed_backup_jobs:
  df.loc[a] = [job, completed_backup_jobs[job]['destinationClient']['clientName'], completed_backup_jobs[job]['appTypeName'], 
                    completed_backup_jobs[job]['sizeOfApplication']/1024/1024/1024, completed_backup_jobs[job]['sizeOfMediaOnDisk']/1024/1024/1024, 
                    completed_backup_jobs[job]['status']]
  a+=1
df_stats = df[['status','jobid']].groupby(['status']).count().reset_index()
df_stats.columns = ['Status','Jobs']
df_stats



---

<br><br>
## **Media Size per Client (Top 10, last 24h)** 

In [0]:
df_stats = df[['client','media_size']].groupby(['client']).sum().reset_index().sort_values(by=['media_size'], ascending=False)
df_stats.columns = ['Client','Media Size (GB)']
df_stats.head(10)



---
<br><br>

##**Media Size per Agent (last 24h)**

In [0]:
df_stats = df[['app_type','media_size']].groupby(['app_type']).sum().reset_index().sort_values(by=['media_size'], ascending=False)
df_stats.columns = ['Application Type','Media Size (GB)']
df_stats



---
<br><br>
## **Properties of randomly selected client**


In [0]:
cls = list(commcell.clients.all_clients.keys())
cltmp = commcell.clients.get(cls[random.randrange(0, len(cls))])
table = BeautifulTable(max_width=300)
table.column_headers = ["Client Name", "Hostname", "Install Dir", 'Data Management Enabled', 'Recovery Enabled', 'Version', 'Service Pack', 'Installed Agents']
table.append_row([cltmp.display_name, cltmp.client_hostname, cltmp.install_directory, cltmp.is_data_management_enabled, cltmp.is_data_recovery_enabled, cltmp.version, cltmp.service_pack, list(cltmp.agents.all_agents.keys())])
print(table)




---
<br><br>

## **Searching a file in backupset backup content:**

In [0]:
FS_CLIENT = 'sqlec2'
FILTER = '*.txt'
clfsbs = commcell.clients.get(FS_CLIENT).agents.get('file system').backupsets.get('defaultbackupset')
paths, properties = clfsbs.find(file_name=FILTER)
print('Found ' + str(len(paths)) + ' ' + FILTER + ' files')
print('')
print('List of files (first 10):')
for ffile in paths[:10]:
  print(ffile)

In [0]:
print('File properties (first file): ' + list(properties.keys())[0])
print()
properties[list(properties.keys())[0]]



---
<br><br>
## **List of protected VMs at VMware client level**


In [0]:
VSA_CLIENT = 'Vcenter_OnPrem'
vsa = commcell.clients.get(VSA_CLIENT).agents.get('virtual server').instances.get('vmware').backupsets.get('defaultbackupset')
table = BeautifulTable(max_width=300)
table.column_headers = ["VM Name", "Size (GB)", "Modified Time"]
ids, vmproperties = vsa.browse()
for vm in vmproperties:
  table.append_row([vmproperties[vm]['name'], vmproperties[vm]['size']/1024/1024/1024, vmproperties[vm]['modified_time']])

print(table)





---
<br><br>
## **List of protected databases at MSSQL client level**


In [0]:
MSSQL_CLIENT = 'sql1'
mssqlbs = commcell.clients.get(MSSQL_CLIENT).agents.get('sql server').instances.get('SQL1\DEMO2017')
table = BeautifulTable(max_width=300)
table.column_headers = ["DB Name", "Last Backup Time"]
paths, properties = mssqlbs.browse()
for db in properties:
  table.append_row([list(db.keys())[0], db[list(db.keys())[0]][0]])

print(table)



---


<br><br>
## **List of Plans**


In [0]:
table = BeautifulTable(max_width=300)
table.column_headers = ["Plan Name", "RPO(h)"]
for plan in commcell.plans.all_plans:
  plantmp = commcell.plans.get(plan)
  table.append_row([plan, plantmp.sla_in_minutes/60])
print(table)

  




---
<br><br>
## **List of Tenants (Companies)**


In [0]:
table = BeautifulTable(max_width=300)
table.column_headers = ["Tenant Name", "Assigned Plans", "Number of Servers", 'Auto Discovery Enabled', 'Auth Code Enabled', 'Tenant Admin']

for org in commcell.organizations.all_organizations:
  orgtmp = commcell.organizations.get(org)
  table.append_row([org, orgtmp.plans, orgtmp.machine_count, orgtmp.is_auto_discover_enabled, orgtmp.is_auth_code_enabled, orgtmp.contacts])
print(table)



---
<br><br>
## **Chargeback Details report - Front End Backup Size (GB)**


In [0]:
url = 'https://' + WEBCONSOLE_URL + '/adminconsole/api/cr/reportsplusengine/datasets/05ea2bd3-ccc0-414f-aeb8-c594124a6855:15a29162-c36a-4267-8c22-8e454aa05522/data/?cacheId=75a2f1db-8a18-4dea-a827-be1508a8928e&offset=0&isExport=false&componentName=Chargeback Details&parameter.TimeRangeType=2&parameter.TimeRangeStart=2147483647&parameter.SizeUnit=GB&parameter.ExcludeDeconfiguredSubclients=No&parameter.ExcludeBackupDisabledSubclients=No&parameter.datasource=2&limit=10000&rawData=false'
payload  = {}
authtoken = commcell.auth_token
headers = {
  'Accept': 'application/json',
  'Authtoken': authtoken
}

response = requests.request("GET", url, headers=headers, data=payload)
report_data = response.json()
report_data['records']

df = pd.DataFrame(columns=['client', 'frontend_backup'])
a = 0
for cl in report_data['records']:
  df.loc[a] = [cl[4], cl[11]]
  a+=1
df_stats = df.groupby(['client']).sum().reset_index().sort_values(by=['frontend_backup'], ascending=False)
df_stats.columns = ['Client','Front End Backup Size (GB)']
df_stats.head(30)




---
<br><br>
## **VM Backup Report**


In [0]:
url = 'https://' + WEBCONSOLE_URL + '/adminconsole/api/cr/reportsplusengine/datasets/D7F9B43F-34A5-45B2-82BA-056EA52FCB77:ae133b83-c1c9-41b1-aa86-f27283e5d633/data/?cacheId=317a9fe9-3062-46bf-907a-9856c436db72&offset=0&isExport=false&componentName=Details&parameter.timerange=All&parameter.LicensingStatus=Off&parameter.datasource=2&limit=200&rawData=false'
payload  = {}
authtoken = commcell.auth_token
headers = {
  'Accept': 'application/json',
  'Authtoken': authtoken
}

response = requests.request("GET", url, headers=headers, data=payload)
report_data = response.json()
df = pd.DataFrame(report_data['records']).loc[:,[3,7,15,18]]
df.columns = ['VM', 'Hypervisor', 'Last Successful Backup', 'Backup Size (GB)']
df.sort_values(by=['Backup Size (GB)'], ascending=False)